# Preparing Management Accounts
Starting with a YTD TTB you need to add the begining of the year trial balance.    

Note that during year during the first months we will not have the accountants adjustments and at some point we will.  At that point there will be a discontinuity in the accounting record.   

If you are preparing part way through the year then you may take a mid year point as the starting point for the preperation of the accounts.  This happend in 2014-2015 and September 2015 was used as a starting point or the subsequent months of the year.

In the first instance this starting point would be the January of the preceding financial year.


## Get the Jan 15 accounts

In [1]:
import datetime as dt
import math
import pandas as pd
import pickle
import sqlite3

from luca import SageData, TransactionalTrialBalance, p, PeriodReport, ExcelManagementReport

In [2]:
rep = PeriodReport(dt.datetime(2015, 1, 1), "historic_trial_balances.db")

In [3]:
emr = ExcelManagementReport()
emr.create_one_report(rep)
rep.prior_mtd.tail()

,TB
Code,
8430,397.45
8433,0.00
8435,1560.49
8440,1877.36
9999,0.00


In [4]:
for attr in ('period_mtd', 'period_ytd', 'prior_mtd', 'prior_ytd'):
    print('Sum for {} = {}'.format(attr, getattr(rep, attr).sum()[0]))

Sum for period_mtd = -75044.01
Sum for period_ytd = 0.0
Sum for prior_mtd = -107345.59
Sum for prior_ytd = 0.0


## Convert YTD for prior year to start of year
We need to close out the year by clearning the P&L and adding to the retained profit for the year.  (Note this doesn't have the accountants changes which we will need for Jan 16).   
rep.period_ytd  --> to year_start.   
Sum > 3000 and add to retained profit 2125
Clear all > 3000

In [5]:
def close_year(df):
    a = df
    pnl = p(a[a.index > 3000].sum()[0])
    print("P&L = {}".format(pnl))
    old_pnl = a.ix[2125][0]
    print("Prev retain = {}, changing to {}".format(old_pnl, old_pnl+pnl))
    b = a.copy()
    b.ix[2125] = old_pnl+pnl
    new_pnl = b.ix[2125][0]
    print("New retained profit = {}".format(new_pnl))
    print("Sum before clearing old balances = {}".format(b.sum()[0]))
    b[b.index > 3000] = p(0)
    print("Sum after clearing old balances = {}".format(b.sum()[0]))
    return b
start_year=close_year(rep.period_ytd)
start_year.tail()

P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.0


,TB
Code,
2109,15151.58
2110,0.00
2120,-100.00
2125,-555654.99
1001,362381.70


## Add start of year to TTB
In order to create the accounts for the YTD we need to add the TTB for the YTD to the year start.   
This should be a generalised TB add.

In [6]:
sage_2015 = pickle.load(open('sage2015.pickle', 'rb'))

In [7]:
ttb = sage_2015.transactional_trial_balance(dt.date(2015, 2, 1), dt.date(2015, 2, 28))
print('Sum ttb {}'.format(ttb.sum()))
ttb.head()

Sum ttb 0.00


ACCOUNT_REF
1001     25646.21
1100     60958.94
1120       468.84
1200    -70576.88
1205         0.00
dtype: object

In [9]:
r = 0
def add_tb(year_open, ttb):
    
    def fill(nc):
        try:
            y_value = ttb[nc]
        except:
            y_value=p(0)
        try:
            t_value = year_open.ix[nc]['TB']
        except:
            t_value = p(0)
        return y_value + t_value
    
    # Create union of nominal codes
    year_nc = list(year_open.index.values)
    ttb_nc = list(ttb.axes[0])
    nc_list = list(set(ttb_nc) | set(year_nc))
    # For each nominal code add from both
    print('Added trial balance')
    nc_list.sort()
    df = pd.DataFrame([p(0)]*len(nc_list), index = nc_list, columns=['TB'])
    df['TB'] = [fill(r) for r in df.index]
    return df

rep_month = PeriodReport(dt.datetime(2015, 1, 1), "historic_trial_balances.db")

ytd = add_tb(start_year, ttb)
print('Sum of new YTD = {}'.format(ytd['TB'].sum()))  # Has year start and YTD added together
ytd.head()

Added trial balance
Sum of new YTD = 0.00


,TB
10,9926.00
1001,388027.91
1100,364415.74
1102,0.00
1103,-30274.58


In [29]:
ytd.ix[[2107, 2210, 2211]]

,TB
2107,-4606.31
2210,122.18
2211,156.59


# Create account for the month
Having created YTD and MTD then upload into the database under temporary notation
Then load a report with the correct data and run the report.

In [30]:
class LoadDatabaseError(Exception):
    pass

class LoadDatabase():
    """Does not have the ability to create a database or chart of acconts if they don't exist."""
    
    def __init__(self, dbname):
        self.conn = sqlite3.connect(dbname)
        self.cursor = self.conn.cursor()
        
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.commit()
        self.close()
 
    def commit(self):
        self.conn.commit()

    def close(self):
        self.conn.close()
 
    def empty(self, period):
        """Check if no data for Balance sheet period is in database"""
        count=self.cursor.execute("SELECT COUNT(*) FROM trial_balance WHERE period='{}' and code < 4000".
                                  format(period)).fetchone()[0]
        return count==0

    def get_coa(self, coa):
        sql = "SELECT code as Code, name as NC_Name, category as Category  FROM chart_of_accounts WHERE chart = '{}'".format(coa)
        return  pd.read_sql(sql, self.conn, index_col='Code')

    
    def load_tb_to_database(self, df, period, overwrite = False):
        """Assume the data already has the correct sign"""
        self.df = df
        if self.empty(period) or overwrite:
            if overwrite:
                    self.cursor.execute("DELETE FROM trial_balance WHERE period = '{}'".format(period))                
            coa=self.get_coa('SLF-MA')
            s = df['TB']
            l = s.index.tolist()
            #print(l)
            for i,v in enumerate(s):
                code = str(int(l[i]))
                # print(l[i], type(l[i]), str(l[i]), v)
                # Insert a row of data
                if v == '-' or math.isnan(v):
                    v = p(0)
                else:
                    v = p(v)
                process_normally = not( ((code=='5001') and (v == p(0)))  # leave out end of year adjustments unless needed
                                      or (code == '2126'))  # Leave out YTD carried forwad P&L which is done from the trial balance
                if process_normally:
                    category = coa.ix[int(code)].Category
                    self.cursor.execute("INSERT INTO trial_balance (period, code, balance) VALUES ('{}', {}, {})".format(
                            period, code, v))
        else:
            raise LoadDatabaseError('{} already is in managegment report database'.format(period))

In [31]:
ttb_converter = TransactionalTrialBalance()
with LoadDatabase('historic_trial_balances.db') as ld:
    coa=ld.get_coa('SLF-MA')
    coa.sort_index()
    # print(coa)
    mgmt_tb=ttb_converter.convert_trial_balance(ytd)
    ld.load_tb_to_database(mgmt_tb, 'AYTD-FEB-15', overwrite=True)
    # TODO the signs of this are wrong it should be negative for 2107 so that in the BS
    # it comes out as positive

In [32]:
new_rep = PeriodReport(dt.datetime(2015, 2, 1), "historic_trial_balances.db", period_ytd_prefix="AYTD")

In [33]:
emr.create_one_report(new_rep)
new_rep.prior_mtd.tail()

,TB
Code,
8430,678.98
8433,0.00
8435,311.91
8440,860.04
9999,0.00


In [34]:
print(mgmt_tb['TB'].sum())
print(mgmt_tb.ix[10])

-52297.94
TB    9926.00
Name: 10, dtype: object


## Create accounts for the year
Iterate for the whole year.

## Using Jan 16 accounts generate accounts for 2016-2017
This can be redone once we have finalised the Jan 16 accounts

# Playpit


In [35]:
new_rep.period_ytd['TB']

Code
10         9926.00
1001     385051.72
1100     364415.74
1102          0.00
1103     -29805.74
1115      65539.72
1200      56886.11
1202     -12178.72
1203          0.00
1204        856.64
2100     -81293.82
2103    -235301.85
2104      24214.33
2105          0.00
2106          0.00
2107      -4327.54
2108      -6742.16
2109       6497.86
2110     -27952.05
2120       -100.00
2125    -555654.99
4000    -156580.95
4009       1209.13
4905       5171.06
5000      91169.17
6100       1430.99
6200       1569.50
6201       1325.00
7000       9347.17
7006          0.00
7020          0.00
7100        750.00
7102       3131.62
7103        194.45
7105        982.83
7200         35.00
7202        920.00
7204       7500.00
7206       1331.01
7906          0.00
8100          0.00
8200        300.00
8201          0.00
8204        150.23
8300        250.00
8310          0.00
8400        125.00
8402        564.68
8405        365.00
8408        168.07
8410       2020.00
8414        260.00
8420   